In [1]:
import os
cwd = os.getcwd()

In [2]:
cd {cwd}

/mnt/File/shamgholi/projects/person_reid/AP3D


# load model

In [3]:
import sys
import os
sys.path.append(cwd)
import time
import math
import os.path as osp
import numpy as np

import torch
from torch.utils.data import DataLoader

from config import Config
import models
import transforms.spatial_transforms as ST
import transforms.temporal_transforms as TT
import tools.data_manager as data_manager
from tools.video_loader import VideoDataset
from tools.utils import Logger
from tools.eval_metrics import evaluate
from commons import modify_model
from tqdm import tqdm
import random


def seed_everythings():
    seed = 1
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)



class Args:
    #"/home/hadi/iust/datasets"
    #"/home/shamgholi/iust/datasets" 
    #"/mnt/File/shamgholi/datasets"
    root = "/mnt/File/shamgholi/datasets"
    dataset = "mars"
    workers = 0
    height = 256
    width = 128
    # test_frames = 8
    arch = "ap3dres50"
    resume = "./"
    pretrain = os.path.join(cwd, "logs/row41/best_model.pth.tar")
    test_epochs = [240]
    distance = "cosine" #cosine
    gpu = "0"
    test_batch = 6
    

In [4]:

# def extract(
#     model, vids, use_gpu):
#     n, c, f, h, w = vids.size()
#     assert(n == 1)

#     if use_gpu:
#         feat = torch.cuda.FloatTensor()
#     else:
#         feat = torch.FloatTensor()
#     for i in range(math.ceil(f/args.test_frames)):
#         clip = vids[:, :, i*args.test_frames:(i+1)*args.test_frames, :, :]
#         print('clip.shape', clip.shape)
#         if use_gpu:
#             clip = clip.cuda()
#         output = model(clip)
#         feat = torch.cat((feat, output), 1)

#     feat = feat.mean(1)
#     feat = model.bn(feat)
#     feat = feat.data.cpu()

#     return feat


def test(model, queryloader, galleryloader, use_gpu, args):
    # test using 4 frames
    since = time.time()
    model.eval()

    qf, q_pids, q_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(queryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        feat = model.bn(feat)
        feat = feat.data.cpu()

        qf.append(feat)
        q_pids.extend(pids)
        q_camids.extend(camids)

    qf = torch.cat(qf, 0)
    q_pids = np.asarray(q_pids)
    q_camids = np.asarray(q_camids)
    print("Extracted features for query set, obtained {} matrix".format(qf.shape))

    gf, g_pids, g_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(galleryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        feat = model.bn(feat)
        feat = feat.data.cpu()

        gf.append(feat)
        g_pids.extend(pids)
        g_camids.extend(camids)

    gf = torch.cat(gf, 0)
    g_pids = np.asarray(g_pids)
    g_camids = np.asarray(g_camids)

    if args.dataset == 'mars':
        # gallery set must contain query set, otherwise 140 query imgs will not have ground truth.
        gf = torch.cat((qf, gf), 0)
        g_pids = np.append(q_pids, g_pids)
        g_camids = np.append(q_camids, g_camids)

    print("Extracted features for gallery set, obtained {} matrix".format(gf.shape))

    time_elapsed = time.time() - since
    print('Extracting features complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print("Computing distance matrix")
    m, n = qf.size(0), gf.size(0)
    distmat = torch.zeros((m,n))

    if args.distance == 'euclidean':
        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        for i in range(m):
            distmat[i:i+1].addmm_(1, -2, qf[i:i+1], gf.t())
    else:
        q_norm = torch.norm(qf, p=2, dim=1, keepdim=True)
        g_norm = torch.norm(gf, p=2, dim=1, keepdim=True)
        qf = qf.div(q_norm.expand_as(qf))
        gf = gf.div(g_norm.expand_as(gf))
        for i in range(m):
            distmat[i] = - torch.mm(qf[i:i+1], gf.t())
    distmat = distmat.numpy()
    
    print("Computing CMC and mAP")
    cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)

    print("Results ----------")
    print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
    print("------------------")

    return cmc[0]

In [7]:
# Data augmentation
args = Args()
conf = Config()
conf.print_model_parameters_trainable = False

dataset = data_manager.init_dataset(name=args.dataset, root=args.root)
spatial_transform_test = ST.Compose([
            ST.Scale((args.height, args.width), interpolation=3),
            ST.ToTensor(),
            ST.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
temporal_transform_test = TT.TemporalBeginCrop()

use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
pin_memory = True if use_gpu else False

queryloader = DataLoader(
    VideoDataset(dataset.query, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)

galleryloader = DataLoader(
    VideoDataset(dataset.gallery, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)



def experiment():
    print("Initializing model: {}".format(args.arch))
    seed_everythings()
    model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
    print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
    model = model.to(device)
    model = model.eval()
    modify_model(model, args, conf)
    with torch.no_grad():
        test(model, queryloader, galleryloader, use_gpu, args)



=> MARS loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # tracklets
  ------------------------------
  train    |   625 |     8298
  query    |   626 |     1980
  gallery  |   622 |     9330
  ------------------------------
  total    |  1247 |    19608
  number of images per tracklet: 2 ~ 920, average 59.5
  ------------------------------


In [9]:
conf.use_linear_to_get_important_features = False
conf.use_linear_to_merge_features = False
conf.use_hist = False
args.dataset = 'logs/row41/best_model.pth.tar'
experiment()

Initializing model: ap3dres50
Model size: 11.75930M
pretrain state dict loaded
----------
model layers:
ResNet503D(
  (conv1): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=(1, 1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck3D(
      (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): Bottleneck3D(
      (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3